In [39]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from time import sleep
from unidecode import unidecode
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [41]:
def get_data(page_source):
    soup = BeautifulSoup(page_source, features='html.parser')
    rows = soup.findAll('tr')[1:]
    rows_data = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]
    return rows_data

In [42]:
service = Service('/opt/homebrew/Caskroom/chromedriver/127.0.6533.88/chromedriver-mac-arm64/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument("no-sandbox");

In [10]:
# Returns the W/L records in Regular Season and Playoffs
def get_coaching_stats(coach_name):
    driver = webdriver.Chrome(service=service, options=options)
    driver.get('https://pro-football-history.com/search_amp_page?coaches=&franchises=')
    #sidebar_button = driver.find_element(By.CLASS_NAME, 'ampstart-navbar-trigger')
    sidebar_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "ampstart-navbar-trigger")))
    sidebar_button.click()
    #entry = driver.find_element(By.NAME, 'search')
    entry = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, "search")))
    entry.click()
    entry.send_keys(coach_name)
    #search_button = driver.find_element(By.CLASS_NAME, 'ampstart-btn')
    search_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "ampstart-btn")))
    sleep(5)
    search_button.click()
    sleep(5)
    page_source = driver.page_source
    stats = get_data(page_source)
    driver.quit()
    if stats == []:
        return [None,None,None,None,None,None]
    return [int(i) for i in stats[0][1].split('-')[:2]] + [round(float(stats[0][2][:-1])/100, 3)] + [int(i) for i in stats[0][3].split('-')] + [round(float(stats[0][4][:-1])/100, 3)]

In [47]:
def get_coach_url(coach_name):
    driver = webdriver.Chrome(service=service, options=options)
    driver.get('https://pro-football-history.com/search_amp_page?coaches=&franchises=')
    #sidebar_button = driver.find_element(By.CLASS_NAME, 'ampstart-navbar-trigger')
    sidebar_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "ampstart-navbar-trigger")))
    sidebar_button.click()
    #entry = driver.find_element(By.NAME, 'search')
    entry = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, "search")))
    entry.click()
    entry.send_keys(coach_name)
    #search_button = driver.find_element(By.CLASS_NAME, 'ampstart-btn')
    search_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "ampstart-btn")))
    sleep(5)
    search_button.click()
    sleep(5)
    page_source = driver.page_source
    stats = get_data(page_source)
    url = driver.current_url
    driver.quit()
    if stats == []:
        return None
    return url

In [129]:
get_coaching_stats('Wes Welker')

[64, 51, 0.556, 4, 5, 0.444]

In [44]:
data = pd.read_csv('phase2data.csv')
coaches = data['Name'].values.tolist()
coaches[49]

'Curtis, Andre'

In [48]:
all_urls = []
for coach in coaches:
    if ',' in coach:
        coach = unidecode(coach.split(',')[1].strip() + ' ' + coach.split(',')[0])
    url = get_coach_url(coach)
    print(coach, url)
    all_urls.append(url)

Gary Brown https://pro-football-history.com/coach/861/gary-brown-bio
Tony Levine https://pro-football-history.com/coach/1177/tony-levine-bio
Rip Scherer https://pro-football-history.com/coach/1196/rip-scherer-bio
Tim Drevno https://pro-football-history.com/coach/1315/tim-drevno-bio
Connor Senger https://pro-football-history.com/coach/13872/connor-senger-bio
Johnny Roland https://pro-football-history.com/coach/1125/johnny-roland-bio
Steve Saunders https://pro-football-history.com/coach/2793/steve-saunders-bio
Fernando Lovo https://pro-football-history.com/coach/13755/fernando-lovo-bio
Stephen Adegoke https://pro-football-history.com/coach/13893/stephen-adegoke-bio
Brian Billick https://pro-football-history.com/coach/32/brian-billick-bio
Marc Colombo https://pro-football-history.com/coach/2393/marc-colombo-bio
Chuck Knox None
Alan Williams https://pro-football-history.com/coach/1215/alan-williams-bio
Chris Watt https://pro-football-history.com/coach/14055/chris-watt-bio
Keith Heyward htt

In [49]:
len(all_urls)

2278

In [51]:
pd.DataFrame(all_urls, columns=['URL']).to_csv('phase2data_urls.csv')

In [21]:
all_stats = []
for coach in coaches:
    if ',' in coach:
        coach = unidecode(coach.split(',')[1].strip() + ' ' + coach.split(',')[0])
    stats = get_coaching_stats(coach)
    print(coach, stats)
    all_stats.append(stats)

Gary Brown [83, 93, 0.472, 2, 3, 0.4]
Tony Levine [15, 17, 0.469, 0, 0, 0.0]
Rip Scherer [58, 86, 0.403, 1, 1, 0.5]
Tim Drevno [36, 11, 0.76, 5, 3, 0.625]
Connor Senger [8, 26, 0.235, 0, 0, 0.0]
Johnny Roland [205, 220, 0.482, 7, 9, 0.438]
Steve Saunders [70, 44, 0.614, 1, 4, 0.2]
Fernando Lovo [3, 14, 0.176, 0, 0, 0.0]
Stephen Adegoke [23, 11, 0.677, 3, 2, 0.6]
Brian Billick [159, 129, 0.552, 7, 9, 0.438]
Marc Colombo [50, 46, 0.521, 1, 2, 0.333]
Chuck Knox [None, None, None, None, None, None]
Alan Williams [211, 144, 0.594, 10, 14, 0.417]
Chris Watt [9, 8, 0.529, 0, 0, 0.0]
Keith Heyward [8, 9, 0.471, 0, 0, 0.0]
Kevin Petry [88, 59, 0.599, 3, 4, 0.429]
Kodi Burns [16, 18, 0.471, 0, 0, 0.0]
Urban Meyer [2, 11, 0.154, 0, 0, 0.0]
Jerry Holmes [25, 55, 0.312, 0, 0, 0.0]
Andrew Hayes-Stoker [108, 87, 0.554, 9, 5, 0.643]
Bruce DeHaven [268, 210, 0.56, 16, 18, 0.471]
Irv Eatman [63, 65, 0.492, 0, 1, 0.0]
Mike Munchak [238, 210, 0.531, 8, 10, 0.444]
Don Shumpert [33, 48, 0.409, 0, 1, 0.0]
To

In [18]:
pd.DataFrame(all_stats, columns=['W','L','%','P_W','P_L','P_%']).to_csv('phase2data_addons2.csv')

In [52]:
urls_df = pd.DataFrame(all_urls, columns=['URL'])

In [22]:
len(all_stats)

50

In [25]:
top_stats = pd.read_csv('phase2data_addons.csv', index_col=0)
top_stats

,W,L,%,P_W,P_L,P_%
0,79.0,97.0,0.449,5.0,4.0,0.556
1,32.0,32.0,0.500,1.0,1.0,0.500
2,199.0,219.0,0.476,4.0,9.0,0.308
3,NaN,NaN,NaN,NaN,NaN,NaN
4,180.0,247.0,0.422,6.0,5.0,0.545
...,...,...,...,...,...,...
2223,129.0,161.0,0.445,5.0,6.0,0.455
2224,173.0,166.0,0.510,6.0,7.0,0.461
2225,9.0,8.0,0.529,1.0,1.0,0.500
2226,48.0,34.0,0.584,0.0,3.0,0.000


In [23]:
all_stats_df = pd.DataFrame(all_stats, columns=['W','L','%','P_W','P_L','P_%'])
all_stats_df

,W,L,%,P_W,P_L,P_%
0,83.0,93.0,0.472,2.0,3.0,0.400
1,15.0,17.0,0.469,0.0,0.0,0.000
2,58.0,86.0,0.403,1.0,1.0,0.500
3,36.0,11.0,0.760,5.0,3.0,0.625
4,8.0,26.0,0.235,0.0,0.0,0.000
5,205.0,220.0,0.482,7.0,9.0,0.438
6,70.0,44.0,0.614,1.0,4.0,0.200
7,3.0,14.0,0.176,0.0,0.0,0.000
8,23.0,11.0,0.677,3.0,2.0,0.600
9,159.0,129.0,0.552,7.0,9.0,0.438


In [26]:
cume_stats = pd.concat([all_stats_df, top_stats])

In [27]:
cume_stats

,W,L,%,P_W,P_L,P_%
0,83.0,93.0,0.472,2.0,3.0,0.400
1,15.0,17.0,0.469,0.0,0.0,0.000
2,58.0,86.0,0.403,1.0,1.0,0.500
3,36.0,11.0,0.760,5.0,3.0,0.625
4,8.0,26.0,0.235,0.0,0.0,0.000
...,...,...,...,...,...,...
2223,129.0,161.0,0.445,5.0,6.0,0.455
2224,173.0,166.0,0.510,6.0,7.0,0.461
2225,9.0,8.0,0.529,1.0,1.0,0.500
2226,48.0,34.0,0.584,0.0,3.0,0.000


In [28]:
cume_stats.to_csv('phase2data_addons2.csv')

In [29]:
data

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC
0,174258.0,"Brown, Gary",11.0,6.0,7/1/1969 0:00:00,8.0,3.0,RB,0.0,Off,NaN
1,660434.0,"Levine, Tony",2.0,18.0,10/28/1972 0:00:00,0.0,3.0,OC,0.0,ST,NaN
2,580829.0,"Scherer, Rip",9.0,30.0,8/3/1952 0:00:00,0.0,3.0,HC,0.0,Off,NaN
3,154631.0,"Drevno, Tim",3.0,29.0,3/20/1969 0:00:00,0.0,3.0,QC,0.0,Off,NaN
4,926678.0,"Senger, Connor",2.0,5.0,NaN,0.0,3.0,OC,0.0,Off,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2273,499900.0,"Skipper, Kelly",17.0,18.0,7/25/1967 0:00:00,0.0,3.0,OC,0.0,Off,NaN
2274,470114.0,"Rodgers, Kacy",21.0,9.0,6/24/1969 0:00:00,0.0,2.0,AH,4.0,Def,NaN
2275,881149.0,"Lester, Tim",1.0,22.0,2/8/1977 0:00:00,0.0,3.0,HC,0.0,Off,NaN
2276,135716.0,"Faulkner, Eddie",5.0,17.0,10/20/1977 0:00:00,1.0,3.0,OC,0.0,NaN,NaN


In [34]:
tot_stats = pd.concat([data.reset_index(drop=True), cume_stats.reset_index(drop=True)], axis=1)

In [35]:
tot_stats

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,W,L,%,P_W,P_L,P_%
0,174258.0,"Brown, Gary",11.0,6.0,7/1/1969 0:00:00,8.0,3.0,RB,0.0,Off,NaN,83.0,93.0,0.472,2.0,3.0,0.400
1,660434.0,"Levine, Tony",2.0,18.0,10/28/1972 0:00:00,0.0,3.0,OC,0.0,ST,NaN,15.0,17.0,0.469,0.0,0.0,0.000
2,580829.0,"Scherer, Rip",9.0,30.0,8/3/1952 0:00:00,0.0,3.0,HC,0.0,Off,NaN,58.0,86.0,0.403,1.0,1.0,0.500
3,154631.0,"Drevno, Tim",3.0,29.0,3/20/1969 0:00:00,0.0,3.0,QC,0.0,Off,NaN,36.0,11.0,0.760,5.0,3.0,0.625
4,926678.0,"Senger, Connor",2.0,5.0,NaN,0.0,3.0,OC,0.0,Off,NaN,8.0,26.0,0.235,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,499900.0,"Skipper, Kelly",17.0,18.0,7/25/1967 0:00:00,0.0,3.0,OC,0.0,Off,NaN,129.0,161.0,0.445,5.0,6.0,0.455
2274,470114.0,"Rodgers, Kacy",21.0,9.0,6/24/1969 0:00:00,0.0,2.0,AH,4.0,Def,NaN,173.0,166.0,0.510,6.0,7.0,0.461
2275,881149.0,"Lester, Tim",1.0,22.0,2/8/1977 0:00:00,0.0,3.0,HC,0.0,Off,NaN,9.0,8.0,0.529,1.0,1.0,0.500
2276,135716.0,"Faulkner, Eddie",5.0,17.0,10/20/1977 0:00:00,1.0,3.0,OC,0.0,NaN,NaN,48.0,34.0,0.584,0.0,3.0,0.000


In [36]:
tot_stats.to_csv('phase2data_edited.csv')

In [53]:
tot_stats_with_url = pd.concat([tot_stats.reset_index(drop=True), urls_df.reset_index(drop=True)], axis=1)
tot_stats_with_url

,CoachID,Name,NFL Coach Experience (thru 2023),College/Other Exp (thru 2023),Date of Birth,NFL Playing Exp,"Highest Pos NFL (1=HC, 2=Coord, 3=ST, Pos or Lower)",Highest Pos College / other Pro,# seasons as NFL Coordinator,Off/Def side of ball,1st Season as NFL HC,W,L,%,P_W,P_L,P_%,URL
0,174258.0,"Brown, Gary",11.0,6.0,7/1/1969 0:00:00,8.0,3.0,RB,0.0,Off,NaN,83.0,93.0,0.472,2.0,3.0,0.400,https://pro-football-history.com/coach/861/gar...
1,660434.0,"Levine, Tony",2.0,18.0,10/28/1972 0:00:00,0.0,3.0,OC,0.0,ST,NaN,15.0,17.0,0.469,0.0,0.0,0.000,https://pro-football-history.com/coach/1177/to...
2,580829.0,"Scherer, Rip",9.0,30.0,8/3/1952 0:00:00,0.0,3.0,HC,0.0,Off,NaN,58.0,86.0,0.403,1.0,1.0,0.500,https://pro-football-history.com/coach/1196/ri...
3,154631.0,"Drevno, Tim",3.0,29.0,3/20/1969 0:00:00,0.0,3.0,QC,0.0,Off,NaN,36.0,11.0,0.760,5.0,3.0,0.625,https://pro-football-history.com/coach/1315/ti...
4,926678.0,"Senger, Connor",2.0,5.0,NaN,0.0,3.0,OC,0.0,Off,NaN,8.0,26.0,0.235,0.0,0.0,0.000,https://pro-football-history.com/coach/13872/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,499900.0,"Skipper, Kelly",17.0,18.0,7/25/1967 0:00:00,0.0,3.0,OC,0.0,Off,NaN,129.0,161.0,0.445,5.0,6.0,0.455,https://pro-football-history.com/coach/2259/ke...
2274,470114.0,"Rodgers, Kacy",21.0,9.0,6/24/1969 0:00:00,0.0,2.0,AH,4.0,Def,NaN,173.0,166.0,0.510,6.0,7.0,0.461,https://pro-football-history.com/coach/841/kac...
2275,881149.0,"Lester, Tim",1.0,22.0,2/8/1977 0:00:00,0.0,3.0,HC,0.0,Off,NaN,9.0,8.0,0.529,1.0,1.0,0.500,https://pro-football-history.com/coach/14053/t...
2276,135716.0,"Faulkner, Eddie",5.0,17.0,10/20/1977 0:00:00,1.0,3.0,OC,0.0,NaN,NaN,48.0,34.0,0.584,0.0,3.0,0.000,https://pro-football-history.com/coach/12683/e...


In [54]:
tot_stats_with_url.to_csv('phase2data_edited.csv')